# Fine-tune Gemma-2-2b for Code Audit (Kaggle T4)

**Production Notebook** | Amit Rosen | Ashkelon, Israel | 2026

| Setting | Value |
|---------|-------|
| Model | `unsloth/gemma-2-2b-it-bnb-4bit` |
| GPU | Tesla T4 (16GB VRAM) |
| Train Time | ~25 min |

## 1. Install Dependencies

**Run this cell, then RESTART KERNEL before continuing!**

In [ ]:
# KAGGLE UNSLOTH INSTALL - NUMPY 2.x COMPATIBLE
# Run this cell, then RESTART KERNEL!

import subprocess
import sys
import os

def run(cmd, msg):
    print(msg)
    subprocess.run(cmd, shell=True, capture_output=True)

# Step 1: Upgrade PyTorch to 2.5.1
run(
    "pip install -q torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu121",
    "[1/5] Installing PyTorch 2.5.1..."
)

# Step 2: Force reinstall numpy (binary compatible with new torch)
run(
    "pip install -q --force-reinstall numpy",
    "[2/5] Reinstalling numpy (binary fix)..."
)

# Step 3: Install Unsloth with specific unsloth_zoo
run(
    'pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"',
    "[3/5] Installing Unsloth..."
)
run("pip install -q unsloth_zoo", "     + unsloth_zoo...")

# Step 4: Training dependencies
run(
    "pip install -q trl==0.9.6 peft accelerate bitsandbytes datasets",
    "[4/5] Installing training dependencies..."
)

# Step 5: xformers
run("pip install -q xformers==0.0.28.post3", "[5/5] Installing xformers...")

print("\n" + "="*60)
print("[OK] Installation complete!")
print("="*60)
print("\n" + "*"*60)
print("*** IMPORTANT: RESTART KERNEL NOW! ***")
print("*** Then run Cell 2 (skip this cell) ***")
print("*"*60)

## 2. Verify Installation

Run this AFTER kernel restart

In [ ]:
# Verify installation (run after kernel restart)
import torch
import numpy as np

print(f"[OK] torch: {torch.__version__}")
print(f"[OK] numpy: {np.__version__}")
print(f"[OK] CUDA: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"[OK] GPU: {torch.cuda.get_device_name(0)}")
    print(f"[OK] VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Test unsloth import
print("\n[INFO] Testing unsloth import...")
from unsloth import FastLanguageModel
print("[OK] Unsloth imported successfully!")

## 3. Load Model

Loading `gemma-2-2b-it` in 4-bit (~5GB VRAM)

In [ ]:
# Configuration
MODEL_NAME = "unsloth/gemma-2-2b-it-bnb-4bit"
MAX_SEQ_LENGTH = 2048
DTYPE = None
LOAD_IN_4BIT = True

print(f"[INFO] Loading {MODEL_NAME}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=DTYPE,
    load_in_4bit=LOAD_IN_4BIT,
)

print(f"\n[OK] Model loaded!")
print(f"[OK] VRAM: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

## 4. Add LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

print("[OK] LoRA adapters added!")
model.print_trainable_parameters()

## 5. Load Dataset

In [ ]:
from datasets import load_dataset
import glob

# Find dataset
paths = ["/kaggle/input/audit-dataset/audit_dataset.jsonl",
         "/kaggle/input/*/audit_dataset.jsonl",
         "/kaggle/input/*/*.jsonl"]

dataset_path = None
for p in paths:
    m = glob.glob(p)
    if m: dataset_path = m[0]; break

if not dataset_path:
    raise FileNotFoundError("Upload audit_dataset.jsonl to Kaggle Input!")

print(f"[INFO] Loading: {dataset_path}")
dataset = load_dataset("json", data_files=dataset_path, split="train")

# Alpaca format
ALPACA = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS = tokenizer.eos_token

def fmt(ex):
    return {"text": [ALPACA.format(i, o) + EOS for i, o in zip(ex["instruction"], ex["output"])]}

dataset = dataset.map(fmt, batched=True)
print(f"[OK] Loaded {len(dataset)} examples")

## 6. Train (~25 min)

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        output_dir="./outputs",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
    ),
)

print("[INFO] Training... (~25 min)")
stats = trainer.train()
print(f"\n[OK] Done in {stats.metrics['train_runtime']:.0f}s")

## 7. Save Model

In [ ]:
# Test inference
FastLanguageModel.for_inference(model)
test = "Analyze test coverage: 330 files, 5 executable, 0% coverage"
inputs = tokenizer([ALPACA.format(test, "")], return_tensors="pt").to("cuda")
out = model.generate(**inputs, max_new_tokens=256, use_cache=True)
print("[TEST]\n", tokenizer.decode(out[0]))

# Save
model.save_pretrained("audit-gemma-v1")
tokenizer.save_pretrained("audit-gemma-v1")
print("\n[OK] Saved to audit-gemma-v1/")